In [1]:
import mlflow
import pandas as pd
import re
from pprint import pprint
mlflow.set_tracking_uri('http://localhost:5000')

def drop_duplicates_and_zero_NA(results_df):
    results_df = results_df.drop_duplicates(subset=['params.run_hash'])

     # Zeroing NAN and other non-converged results
    results_df.loc[results_df['metrics.l1_match_val'].isna() | (results_df['metrics.test_hits_at_1'] == 1.0), 
                   ['metrics.l1_match_train', 'metrics.l1_match_val',
                   'metrics.train_mr', 'metrics.test_mr', 'metrics.test_hits_at_100',
                   'metrics.train_hits_at_100', 'metrics.train_hits_at_10',
                   'metrics.val_hits_at_10', 'metrics.val_hits_at_100',
                   'metrics.train_hits_at_50', 'metrics.test_hits_at_50', 'metrics.val_mr',
                   'metrics.test_mrr', 'metrics.train_mrr', 'metrics.val_hits_at_1',
                   'metrics.train_hits_at_1', 'metrics.val_mrr', 'metrics.test_hits_at_1',
                   'metrics.test_hits_at_10', 'metrics.val_hits_at_50']] = 0.0
    return results_df

def formar_df(results, new_metrics_names = ['Hits@1','Hits@10','Hits@50'], 
              unstack_levels = [3, 2],
              hyperparam_names = [None, 'Weights'],
              params_names = ['No','Yes'], scale=100, round_digits=2):
    results.columns = results.columns.get_level_values(0)
    results = results*scale
    results = results.round(round_digits)
    results = results.stack(level=0).unstack(level=unstack_levels)
    results.sort_index(1, inplace=True)
    results.columns.set_levels(new_metrics_names, level=0, inplace=True)
    results.columns.set_levels(params_names, level=1, inplace=True)
    results.index.rename(['Dataset', 'Subset'], inplace=True)
    results.columns.rename(hyperparam_names, inplace=True)
    new_index = [(re.sub('_(.+)', r' (\g<1>)', dataset.upper()), re.sub('_', '-', subset)) for (dataset, subset) in results.index]
    results.index = pd.MultiIndex.from_tuples(new_index)
    return results

def to_latex(results):
    print(results.to_latex(bold_rows=True, 
                         column_format=r'*{2}{l}*{6}{@{\extracolsep{\fill}}r}@{\extracolsep{\fill}}'))

In [2]:
# Save all results to csv
results = mlflow.search_runs(experiment_ids=[1])
results = drop_duplicates_and_zero_NA(results)
results.to_csv('gcn-align-results.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## 1. Hyperparameter search for dbp15k_jape (zh_en)
```python
 params_search = {
      'dataset_name': ['dbp15k_jape'],
      'subset_name': ['zh_en'],
      'num_epochs': [10, 500, 2000, 3000],
      'lr': [0.1, 0.5, 1, 10, 20],
      'eval_batch_size': [1000],  # 1000
      'embedding_dim': [200],
      'n_layers': [1, 2, 3],
      'num_negatives': [5, 50, 100],
      'use_edge_weights': [EdgeWeightsEnum.inverse_in_degree],  # None
      'use_conv_weights': [True, False],
      'conv_weight_init': [init.xavier_uniform_],
      'train_val_ratio': [0.7, 0.8],  # size of train subset in comparison with
      'node_embedding_init': ['total', 'none'],
      'optimizer': [optim.SGD, optim.Adam],
      'vertical_sharing': [True, False],
      'seed': [seed]
  }
```

In [3]:
results = mlflow.search_runs(experiment_ids=[1],
                             filter_string = "params.dataset_name = 'dbp15k_jape' and params.subset_name = 'zh_en'")
results = drop_duplicates_and_zero_NA(results)
results = results[results['params.lr'].isin(['0.1', '0.5', '1', '10', '20']) & 
                  results['params.n_layers'].isin(['1', '2', '3']) & 
                  results['params.num_negatives'].isin(['5', '50', '100']) &
                  results['params.num_epochs'].isin(['10', '500', '2000', '3000']) & 
                  results['params.train_val_ratio'].isin(['0.7', '0.8']) &
                  results['params.optimizer'].isin(['Adam', 'SGD'])]
len(results)

3615

In [5]:
parametrised = results[results['params.use_conv_weights'] == 'True']
non_parametrised = results[results['params.use_conv_weights'] == 'False']

### Best models for dbp15k_jape (zh_en)
#### Without weigths

```python
params_search_nonpar = {
        'dataset_name': ['dbp15k_jape'],
        'subset_name': ['zh_en'],
        'num_epochs': [2000],
        'lr': [1],
        'eval_batch_size': [1000],  # 1000
        'embedding_dim': [200],
        'n_layers': [2],
        'num_negatives': [50],
        'use_edge_weights': [EdgeWeightsEnum.inverse_in_degree],  # None
        'use_conv_weights': [False],
        'conv_weight_init': [init.xavier_uniform_],
        'train_val_ratio': [0.8],  # size of train subset in comparison with
        'node_embedding_init': ['none'],  # Cite normalisation constant
        'optimizer': [optim.Adam],
        'seed': [seed]
    }
```

In [6]:
non_parametrised.nlargest(1, 'metrics.val_hits_at_1').iloc[0].filter(regex='params\\..+')

params.n_layers                                               2
params.num_epochs                                          2000
params.use_conv_weights                                   False
params.vertical_sharing                                    None
params.total_params                                     7792000
params.num_negatives                                         50
params.train_val_ratio                                      0.8
params.use_edge_weights       EdgeWeightsEnum.inverse_in_degree
params.embedding_dim                                        200
params.run_hash                0c23b25f180d4a1e0e14dfa92f60af8b
params.node_embedding_init                                 none
params.dataset_name                                 dbp15k_jape
params.conv_weight_init                         xavier_uniform_
params.trainable_params                                 7792000
params.eval_batch_size                                     1000
params.seed                             

In [7]:
non_parametrised.nlargest(5, 'metrics.val_hits_at_1').filter(regex='metrics\\.test.+')

,metrics.test_mrr,metrics.test_hits_at_50,metrics.test_hits_at_100,metrics.test_hits_at_10,metrics.test_mr,metrics.test_hits_at_1
1516,0.506254,0.846952,0.875571,0.727286,273.394165,0.392524
1617,0.504022,0.834762,0.863048,0.717381,336.529877,0.393429
2799,0.465469,0.864333,0.898524,0.710667,234.702316,0.345286
2754,0.463656,0.868952,0.903000,0.709762,211.858170,0.342286
1277,0.497176,0.802667,0.830143,0.699429,487.546844,0.392667


#### With weigths
```python
params_search_par = {
        'dataset_name': ['dbp15k_jape'],
        'subset_name': ['zh_en'],
        'num_epochs': [2000],
        'lr': [1],
        'eval_batch_size': [1000],  # 1000
        'embedding_dim': [200],
        'n_layers': [3],
        'num_negatives': [50],
        'use_edge_weights': [EdgeWeightsEnum.inverse_in_degree],  # None
        'use_conv_weights': [True],
        'vertical_sharing': [False],
        'conv_weight_init': [init.xavier_uniform_],
        'train_val_ratio': [0.8],  # size of train subset in comparison with
        'node_embedding_init': ['none'],  # Cite normalisation constant
        'optimizer': [optim.Adam],
        'seed': [seed]
    }
```

In [8]:
parametrised.nlargest(1, 'metrics.val_hits_at_1').iloc[0].filter(regex='params\\..+')

params.n_layers                                               3
params.num_epochs                                          2000
params.use_conv_weights                                    True
params.vertical_sharing                                   False
params.total_params                                     7792000
params.num_negatives                                         50
params.train_val_ratio                                      0.8
params.use_edge_weights       EdgeWeightsEnum.inverse_in_degree
params.embedding_dim                                        200
params.run_hash                9e3f4677e3186b8483eb22f2cbbcac51
params.node_embedding_init                                 none
params.dataset_name                                 dbp15k_jape
params.conv_weight_init                         xavier_uniform_
params.trainable_params                                 7792000
params.eval_batch_size                                     1000
params.seed                             

In [9]:
parametrised.nlargest(5, 'metrics.val_hits_at_1').filter(regex='metrics\\.test.+')

,metrics.test_mrr,metrics.test_hits_at_50,metrics.test_hits_at_100,metrics.test_hits_at_10,metrics.test_mr,metrics.test_hits_at_1
928,0.432034,0.839476,0.877667,0.680667,228.109589,0.310762
922,0.413043,0.839381,0.883381,0.664952,207.325851,0.290667
924,0.430171,0.800857,0.837190,0.661619,344.432251,0.315429
930,0.447213,0.781571,0.819714,0.659429,368.029846,0.338476
918,0.415737,0.795476,0.835476,0.647524,362.665588,0.299857


### Model, proposed used in the original GCN-Align code:
```
params.dataset_name = 'dbp15k_jape' and params.subset_name = 'zh_en' and params.num_epochs = '2000' and params.lr = '20' and params.eval_batch_size = '1000' and params.embedding_dim = '200' and params.n_layers = '2' and params.num_negatives = '5' and params.use_edge_weights = 'EdgeWeightsEnum.inverse_in_degree' and params.use_conv_weights = 'False' and params.conv_weight_init = 'xavier_uniform_' and params.train_val_ratio = '0.8' and params.node_embedding_init = 'total' and params.optimizer = 'SGD' and params.seed = '12306'

```

In [10]:
non_parametrised[non_parametrised['run_id'] == '32901ef5fe1c4f0f9de036cf4a951434'].filter(regex='metrics\\.test.+')

,metrics.test_mrr,metrics.test_hits_at_50,metrics.test_hits_at_100,metrics.test_hits_at_10,metrics.test_mr,metrics.test_hits_at_1
2670,0.484587,0.834286,0.867381,0.712095,340.532806,0.371524


### Extra - percentage of runs, where not using weights is better

In [11]:
merged = pd.merge(parametrised, non_parametrised, 
         on=['params.seed', 'params.optimizer', 'params.dataset_name', 
             'params.train_val_ratio', 'params.conv_weight_init', 'params.embedding_dim', 
             'params.subset_name', 'params.lr', 'params.node_embedding_init',
             'params.num_negatives', 'params.use_edge_weights', 'params.num_epochs', 'params.n_layers'], 
        suffixes=('_par', '_nonpar'))

In [12]:
(merged['metrics.val_hits_at_1_nonpar'] >= merged['metrics.val_hits_at_1_par']).value_counts(normalize = True)

True     0.74931
False    0.25069
dtype: float64

In [13]:
((merged['metrics.val_hits_at_10_nonpar'] >= merged['metrics.val_hits_at_10_par'])).value_counts(normalize = True)

True     0.714811
False    0.285189
dtype: float64

In [14]:
((merged['metrics.val_hits_at_50_nonpar'] >= merged['metrics.val_hits_at_50_par'])).value_counts(normalize = True)

True     0.702852
False    0.297148
dtype: float64

## 2. Retraining best param and non-param models on all the datasets
`train_val_ratio` is set to `1.0`

In [15]:
results_2 = mlflow.search_runs(experiment_ids=[1],
                             filter_string = "params.train_val_ratio = '1.0'")
len(results_2)

24

In [16]:
results_retrain = results_2.groupby(by=['params.dataset_name', 'params.subset_name', 'params.use_conv_weights']).\
 agg({'metrics.test_hits_at_1' : ['first'], 
      'metrics.test_hits_at_10' : ['first'],
      'metrics.test_hits_at_50' : ['first']})
results_retrain = formar_df(results_retrain)

In [17]:
results_retrain

Hits@1        Hits@10        Hits@50       
Weights                 No    Yes      No    Yes      No    Yes
DBP15K (FULL) fr-en  31.23  25.05   67.17  61.17   81.51  79.50
              ja-en  33.40  28.33   66.41  61.49   80.80  78.86
              zh-en  31.26  25.01   64.73  58.79   78.89  77.25
DBP15K (JAPE) fr-en  44.71  33.83   81.78  73.67   91.68  90.03
              ja-en  45.29  35.85   78.28  73.16   89.00  88.24
              zh-en  42.89  33.86   76.37  71.57   86.70  86.51
DWY100K       wd     58.22  42.31   85.73  74.75   93.48  88.74
              yg     73.00  60.56   92.10  84.73   97.33  94.26
WK3L120K      en-de   9.98   4.78   26.92  16.88   43.80  32.45
              en-fr   8.03   3.20   23.41  11.96   38.99  23.82
WK3L15K       en-de  16.19   5.93   41.94  23.00   66.72  45.77
              en-fr  18.68   8.93   51.50  35.43   72.20  61.93

In [18]:
to_latex(results_retrain)

\begin{tabular}{*{2}{l}*{6}{@{\extracolsep{\fill}}r}@{\extracolsep{\fill}}}
\toprule
        & {} & \multicolumn{2}{l}{Hits@1} & \multicolumn{2}{l}{Hits@10} & \multicolumn{2}{l}{Hits@50} \\
        & \textbf{Weights} &     No &    Yes &      No &    Yes &      No &    Yes \\
\midrule
\textbf{DBP15K (FULL)} & \textbf{fr-en} &  31.23 &  25.05 &   67.17 &  61.17 &   81.51 &  79.50 \\
        & \textbf{ja-en} &  33.40 &  28.33 &   66.41 &  61.49 &   80.80 &  78.86 \\
        & \textbf{zh-en} &  31.26 &  25.01 &   64.73 &  58.79 &   78.89 &  77.25 \\
\textbf{DBP15K (JAPE)} & \textbf{fr-en} &  44.71 &  33.83 &   81.78 &  73.67 &   91.68 &  90.03 \\
        & \textbf{ja-en} &  45.29 &  35.85 &   78.28 &  73.16 &   89.00 &  88.24 \\
        & \textbf{zh-en} &  42.89 &  33.86 &   76.37 &  71.57 &   86.70 &  86.51 \\
\textbf{DWY100K} & \textbf{wd} &  58.22 &  42.31 &   85.73 &  74.75 &   93.48 &  88.74 \\
        & \textbf{yg} &  73.00 &  60.56 &   92.10 &  84.73 &   97.33 &  94.26 \\
\textbf{WK

In [19]:
results_retrain_mrr = results_2.groupby(by=['params.dataset_name', 'params.subset_name', 'params.use_conv_weights']).\
 agg({'metrics.test_mr' : ['first'], 
      'metrics.test_mrr' : ['first']})
results_retrain_mrr = formar_df(results_retrain_mrr, new_metrics_names=['MR', 'MRR'], scale=1, round_digits=3)
results_retrain_mrr

MR              MRR       
Weights                    No       Yes     No    Yes
DBP15K (FULL) fr-en   267.523   257.094  0.431  0.369
              ja-en   276.643   266.306  0.444  0.392
              zh-en   342.393   288.309  0.424  0.361
DBP15K (JAPE) fr-en   139.709   111.004  0.573  0.469
              ja-en   197.384   137.570  0.566  0.482
              zh-en   238.115   180.020  0.543  0.463
DWY100K       wd      514.609   312.208  0.680  0.535
              yg       95.302    59.714  0.798  0.691
WK3L120K      en-de  2844.808  2142.350  0.159  0.091
              en-fr  4331.123  3986.397  0.133  0.063
WK3L15K       en-de   252.060   266.289  0.250  0.119
              en-fr   217.567   172.890  0.297  0.177

In [20]:
to_latex(results_retrain_mrr)

\begin{tabular}{*{2}{l}*{6}{@{\extracolsep{\fill}}r}@{\extracolsep{\fill}}}
\toprule
        & {} & \multicolumn{2}{l}{MR} & \multicolumn{2}{l}{MRR} \\
        & \textbf{Weights} &        No &       Yes &     No &    Yes \\
\midrule
\textbf{DBP15K (FULL)} & \textbf{fr-en} &   267.523 &   257.094 &  0.431 &  0.369 \\
        & \textbf{ja-en} &   276.643 &   266.306 &  0.444 &  0.392 \\
        & \textbf{zh-en} &   342.393 &   288.309 &  0.424 &  0.361 \\
\textbf{DBP15K (JAPE)} & \textbf{fr-en} &   139.709 &   111.004 &  0.573 &  0.469 \\
        & \textbf{ja-en} &   197.384 &   137.570 &  0.566 &  0.482 \\
        & \textbf{zh-en} &   238.115 &   180.020 &  0.543 &  0.463 \\
\textbf{DWY100K} & \textbf{wd} &   514.609 &   312.208 &  0.680 &  0.535 \\
        & \textbf{yg} &    95.302 &    59.714 &  0.798 &  0.691 \\
\textbf{WK3L120K} & \textbf{en-de} &  2844.808 &  2142.350 &  0.159 &  0.091 \\
        & \textbf{en-fr} &  4331.123 &  3986.397 &  0.133 &  0.063 \\
\textbf{WK3L15K} & \text

## 3. Ablation study
### For use_conv_weights == True or (use_conv_weights == False and node_embedding_init == 'none')
```python
subsets = {
    'dwy100k': ['wd', 'yg'],
    'wk3l15k': ['en_de', 'en_fr'],
    'wk3l120k': ['en_de', 'en_fr'],
    'dbp15k_full': ['zh_en', 'ja_en', 'fr_en'],
    'dbp15k_jape': ['ja_en', 'fr_en', 'zh_en'], 
}

params_search = {
    'dataset_name': [args.dataset_name],
    'subset_name': subsets[args.dataset_name],
    'num_epochs': [2000],
    'lr': [1, 10, 20, 30],
    'eval_batch_size': [1000], 
    'embedding_dim': [200],
    'n_layers': [2, 3, 4],
    'vertical_sharing': [False, True],
    'num_negatives': [50],
    'use_edge_weights': [EdgeWeightsEnum.inverse_in_degree],  # None
    'use_conv_weights': [False, True],
    'conv_weight_init': [init.xavier_uniform_],
    'train_val_ratio': [0.8],  # size of train subset in comparison with
    'node_embedding_init': ['total', 'none'], 
    'optimizer': [optim.Adam],
    'seed': [seed]
}
```
 

In [21]:
results = mlflow.search_runs(experiment_ids=[1])
results = drop_duplicates_and_zero_NA(results)
results_3 = results[results['params.lr'].isin(['1', '10', '20', '30']) & 
                  results['params.n_layers'].isin(['2', '3', '4']) & 
                  (results['params.num_negatives'] == '50') &
                  (results['params.num_epochs'] == '2000') & 
                  (results['params.train_val_ratio'] == '0.8') &
                  (results['params.optimizer'] == 'Adam')]
len(results_3)

876

In [22]:
# split_by = ['params.dataset_name', 'params.subset_name', 'params.node_embedding_init']
split_by = ['params.dataset_name', 'params.subset_name', 'params.node_embedding_init', 'params.use_conv_weights']

results_abl_non_weights = results_3.sort_values('metrics.val_hits_at_1', ascending=False).drop_duplicates(split_by)
results_abl_non_weights = results_abl_non_weights[~((results_abl_non_weights['params.node_embedding_init'] == 'total') & 
                                                  (results_abl_non_weights['params.use_conv_weights'] == 'False'))]
results_abl_non_weights = results_abl_non_weights.groupby(by=split_by).\
    agg({'metrics.test_hits_at_1' : ['first'], 
      'metrics.test_hits_at_10' : ['first'],
      'metrics.test_hits_at_50' : ['first']})

In [23]:
results_abl_non_weights_table = formar_df(results_abl_non_weights, unstack_levels = [4, 3, 2],
                                          hyperparam_names = [None, 'Weights', 'Emb. init'])
results_abl_non_weights_table

Hits@1               Hits@10               Hits@50         \
Weights                 No    Yes             No    Yes             No    Yes   
Emb. init             none   none  total    none   none  total    none   none   
DBP15K (FULL) fr-en  27.60  21.09  15.23   63.08  57.89  45.67   79.08  81.25   
              ja-en  30.39  25.75  19.76   64.65  59.61  48.06   81.08  80.93   
              zh-en  27.54  21.68  17.31   62.95  54.76  47.58   81.53  73.27   
DBP15K (JAPE) fr-en  41.44  32.16  26.95   78.81  71.23  65.71   90.10  86.73   
              ja-en  40.61  32.88  27.22   74.37  68.99  62.08   86.50  84.06   
              zh-en  39.25  31.08  26.93   72.73  68.07  62.45   84.70  83.95   
DWY100K       wd     54.39  45.60  33.67   82.62  74.95  64.22   91.64  85.93   
              yg     70.49  64.43  48.00   90.67  86.35  76.03   96.65  93.95   
WK3L120K      en-de   9.12   8.62   6.09   25.36  24.10  19.48   42.13  40.60   
              en-fr   7.55   6.89   4.73   22.34  21.12  15.46   37.67  36.37   
WK3L15K       en-de  15.29  15.79  11.52   40.95  42.09  35.26   65.77  67.71   
              en-fr  15.80  14.43  10.68   47.47  44.05  38.09   69.35  65.32   

                            
Weights                     
Emb. init            total  
DBP15K (FULL) fr-en  68.15  
              ja-en  68.43  
              zh-en  70.18  
DBP15K (JAPE) fr-en  84.33  
              ja-en  79.47  
              zh-en  79.53  
DWY100K       wd     79.27  
              yg     88.64  
WK3L120K      en-de  35.16  
              en-fr  28.15  
WK3L15K       en-de  61.30  
              en-fr  63.48

### For use_conv_weights == False and node_embedding_init == 'total'
```python
subsets = {
    'dwy100k': ['wd', 'yg'],
    'wk3l15k': ['en_de', 'en_fr'],
    'wk3l120k': ['en_de', 'en_fr'],
    'dbp15k_full': ['zh_en', 'ja_en', 'fr_en'],
    'dbp15k_jape': ['ja_en', 'fr_en', 'zh_en'], 
}

params_search = {
    'dataset_name': [args.dataset_name],
    'subset_name': subsets[args.dataset_name],
    'num_epochs': [2000, 3000],
    'lr': [0.5, 1],
    'eval_batch_size': [1000],  # 1000
    'embedding_dim': [200],
    'n_layers': [2],
    'num_negatives': [100],
    'use_edge_weights': [EdgeWeightsEnum.inverse_in_degree],  # None
    'use_conv_weights': [False, True],
    'conv_weight_init': [init.xavier_uniform_],
    'train_val_ratio': [0.8],  # size of train subset in comparison with
    'node_embedding_init': ['total', 'none'], 
    'optimizer': [optim.SGD],
    'seed': [seed]
}
```

In [24]:
results = mlflow.search_runs(experiment_ids=[1])
results = drop_duplicates_and_zero_NA(results)
results_4 = results[results['params.lr'].isin(['0.5', '1']) & 
                  results['params.n_layers'].isin(['2']) & 
                  (results['params.num_negatives'] == '100') &
                  (results['params.num_epochs'].isin(['2000', '3000'])) & 
                  (results['params.train_val_ratio'] == '0.8') &
                  (results['params.optimizer'] == 'SGD') &
                  results['params.node_embedding_init'].isin(['total', 'none'])]
len(results_4)

194

In [25]:
split_by = ['params.dataset_name', 'params.subset_name', 'params.node_embedding_init', 'params.use_conv_weights']


results_abl_weights = results_4.sort_values('metrics.val_hits_at_1', ascending=False).drop_duplicates(split_by)
results_abl_weights = results_abl_weights[((results_abl_weights['params.node_embedding_init'] == 'total') & 
                                                  (results_abl_weights['params.use_conv_weights'] == 'False'))]


results_abl_weights = results_abl_weights.groupby(by=split_by).\
    agg({'metrics.test_hits_at_1' : ['first'], 
      'metrics.test_hits_at_10' : ['first'],
      'metrics.test_hits_at_50' : ['first']})

In [26]:
results_abl_weights_table = formar_df(results_abl_weights, unstack_levels = [4, 3, 2],
                                          hyperparam_names = [None, 'Weights', 'Emb. init'])
results_abl_weights_table

Hits@1 Hits@10 Hits@50
Weights                 No      No      No
Emb. init            total   total   total
DBP15K (FULL) fr-en  23.83   57.60   75.63
              ja-en  25.29   56.30   73.04
              zh-en  22.07   53.26   71.19
DBP15K (JAPE) fr-en  36.96   74.87   88.85
              ja-en  35.12   69.74   83.73
              zh-en  34.44   68.03   81.59
DWY100K       wd     48.54   76.17   87.19
              yg     62.84   84.40   93.09
WK3L120K      en-de   8.29   23.43   39.55
              en-fr   6.63   20.01   34.87
WK3L15K       en-de  13.18   37.14   61.78
              en-fr  14.46   45.18   67.66

In [27]:
# Final ablation table
results_abl = results_abl_non_weights.append(results_abl_weights).sort_index(0)
results_abl = formar_df(results_abl, unstack_levels = [4, 3, 2], hyperparam_names = [None, 'Weights', 'Emb. init'])
results_abl = results_abl.reindex(['total', 'none'], level=2, axis=1)
results_abl

Hits@1                      Hits@10                       \
Weights                 No           Yes             No           Yes          
Emb. init            total   none  total   none   total   none  total   none   
DBP15K (FULL) fr-en  23.83  27.60  15.23  21.09   57.60  63.08  45.67  57.89   
              ja-en  25.29  30.39  19.76  25.75   56.30  64.65  48.06  59.61   
              zh-en  22.07  27.54  17.31  21.68   53.26  62.95  47.58  54.76   
DBP15K (JAPE) fr-en  36.96  41.44  26.95  32.16   74.87  78.81  65.71  71.23   
              ja-en  35.12  40.61  27.22  32.88   69.74  74.37  62.08  68.99   
              zh-en  34.44  39.25  26.93  31.08   68.03  72.73  62.45  68.07   
DWY100K       wd     48.54  54.39  33.67  45.60   76.17  82.62  64.22  74.95   
              yg     62.84  70.49  48.00  64.43   84.40  90.67  76.03  86.35   
WK3L120K      en-de   8.29   9.12   6.09   8.62   23.43  25.36  19.48  24.10   
              en-fr   6.63   7.55   4.73   6.89   20.01  22.34  15.46  21.12   
WK3L15K       en-de  13.18  15.29  11.52  15.79   37.14  40.95  35.26  42.09   
              en-fr  14.46  15.80  10.68  14.43   45.18  47.47  38.09  44.05   

                    Hits@50                       
Weights                  No           Yes         
Emb. init             total   none  total   none  
DBP15K (FULL) fr-en   75.63  79.08  68.15  81.25  
              ja-en   73.04  81.08  68.43  80.93  
              zh-en   71.19  81.53  70.18  73.27  
DBP15K (JAPE) fr-en   88.85  90.10  84.33  86.73  
              ja-en   83.73  86.50  79.47  84.06  
              zh-en   81.59  84.70  79.53  83.95  
DWY100K       wd      87.19  91.64  79.27  85.93  
              yg      93.09  96.65  88.64  93.95  
WK3L120K      en-de   39.55  42.13  35.16  40.60  
              en-fr   34.87  37.67  28.15  36.37  
WK3L15K       en-de   61.78  65.77  61.30  67.71  
              en-fr   67.66  69.35  63.48  65.32

In [28]:
to_latex(results_abl[['Hits@1']])

\begin{tabular}{*{2}{l}*{6}{@{\extracolsep{\fill}}r}@{\extracolsep{\fill}}}
\toprule
        & {} & \multicolumn{4}{l}{Hits@1} \\
        & \textbf{Weights} & \multicolumn{2}{l}{No} & \multicolumn{2}{l}{Yes} \\
        & \textbf{Emb. init} &  total &   none &  total &   none \\
\midrule
\textbf{DBP15K (FULL)} & \textbf{fr-en} &  23.83 &  27.60 &  15.23 &  21.09 \\
        & \textbf{ja-en} &  25.29 &  30.39 &  19.76 &  25.75 \\
        & \textbf{zh-en} &  22.07 &  27.54 &  17.31 &  21.68 \\
\textbf{DBP15K (JAPE)} & \textbf{fr-en} &  36.96 &  41.44 &  26.95 &  32.16 \\
        & \textbf{ja-en} &  35.12 &  40.61 &  27.22 &  32.88 \\
        & \textbf{zh-en} &  34.44 &  39.25 &  26.93 &  31.08 \\
\textbf{DWY100K} & \textbf{wd} &  48.54 &  54.39 &  33.67 &  45.60 \\
        & \textbf{yg} &  62.84 &  70.49 &  48.00 &  64.43 \\
\textbf{WK3L120K} & \textbf{en-de} &   8.29 &   9.12 &   6.09 &   8.62 \\
        & \textbf{en-fr} &   6.63 &   7.55 &   4.73 &   6.89 \\
\textbf{WK3L15K} & \textbf{en